In [1]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>")) # Increase cell width
display(HTML("<style>.rendered_html { font-size: 16px; }</style>")) # Increase font size


# for data manipulation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xlrd
pd.set_option('display.max_columns', 60)
import json
import geocoder
from holidays_es import Province
from datetime import datetime, timedelta


# for visualization
from IPython.core.pylabtools import figsize
from matplotlib import pyplot as plt
%matplotlib inline
# to include graphs inline within the frontends next to code
import seaborn as sns
sns.set_context(font_scale=2)
import folium
from pandas_profiling import ProfileReport


# to bypass warnings in various dataframe assignments
pd.options.mode.chained_assignment = None

# machine learning models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

# preprocessing functions and evaluation models
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from statsmodels.tsa.seasonal import seasonal_decompose

# Clustering
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

C:\Users\Carlos.Blazquez\AppData\Local\Temp\ipykernel_10100\2263695952.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


C:\Users\Carlos.Blazquez\AppData\Local\Temp\ipykernel_10100\2263695952.py:26: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [2]:
######Read checkpoint df_ventas_historic 4 local
#_df_ventas_historic = pd.read_csv('Datasets/_df_ventas_historic_CP4.csv')

######Read checkpoint df_ventas_historic 2 local
_df_ventas_historic_cp2 = pd.read_csv('Datasets/_df_ventas_historic_CP2.csv')

######Read checkpoint macrodata_cleaned
macrodata_cleaned = pd.read_csv('Datasets/Macro_por_tienda.csv')

In [3]:
####Mapeo de store_sq_meters 0 - Store, 1 - little supermarket, 2 - medium supermarket, 3 - big supermarket and 4 - hypermarket
def map_store_sq_meters(value):
    if value <= 99:
        return '0'
    elif value <= 400:
        return '1'
    elif value <= 1000:
        return '2'
    elif value <= 2500:
        return '3'
    else:
        return '4'

In [4]:
#########################################3
###############################NOEJECUTA

_df_ventas_historic = _df_ventas_historic_cp2.copy()

In [5]:
#obtaining only categorical columns to code using one-hot or ordinal encoding.  DISCARD CATEGORY onely one value
_df_ventas_historic['PACKAGING'].unique()

array(['CAN', 'CRISTAL BOTTLE', 'PET', 'OTHER'], dtype=object)

In [6]:
_df_ventas_historic.select_dtypes(include=['object']).columns.tolist()

['month',
 'season',
 'CATEGORY',
 'SEGMENT',
 'MANUFACTURER',
 'BRAND',
 'PACKAGING',
 'VOLUME',
 'UNITS',
 'factor_extrapolacion',
 'Canal',
 'Channel',
 'Provincia',
 'Comunidad autónoma',
 'national_holidays_2021',
 'regional_holidays_2021',
 'local_holidays_2021',
 'national_holidays_2022',
 'regional_holidays_2022',
 'local_holidays_2022',
 'national_holidays_2023',
 'regional_holidays_2023',
 'local_holidays_2023']

In [7]:
_df_ventas_historic['month'].unique()

array(['October', 'May', 'September', 'July', 'December', 'April',
       'February', 'August', 'January', 'March', 'June', 'November'],
      dtype=object)

In [8]:
_df_ventas_historic['season'].unique()

array(['Fall', 'Spring', 'Summer', 'Winter'], dtype=object)

In [9]:
_df_ventas_historic['CATEGORY'].unique()

array(['BEER'], dtype=object)

In [10]:
_df_ventas_historic['SEGMENT'].unique()

array(['EXTRA', 'IMPORT SPECIAL', 'IMPORT PREMIUM', 'NATIONAL', 'BLACK',
       'FLAVOURED', 'WITHOUT ALCOHOL', 'ZERO ALCOHOL', 'ARTISANS',
       'CELIACS'], dtype=object)

In [11]:
_df_ventas_historic['MANUFACTURER'].unique()

array(['MANUFACTURER 2', 'MANUFACTURER 4', 'MANUFACTURER 3',
       'MANUFACTURER 1', 'OTHER MANUFACTURERS', 'MANUFACTURER 5',
       'MANUFACTURER 6'], dtype=object)

In [12]:
_df_ventas_historic['BRAND'].unique()

array(['MAN 2 - BRAND 1', 'MAN 2 - BRAND 2', 'MAN 4 - BRAND 1',
       'MAN 2 - OTHER BRANDS', 'MAN 3 - BRAND 3', 'MAN 3 - BRAND 4',
       'MAN 1 - BRAND 1', 'OTHER MAN - OTHER BRANDS', 'MAN 2 - BRAND 5',
       'MAN 5 - OTHER BRANDS', 'MAN 3 - OTHER BRANDS', 'MAN 2 - BRAND 3',
       'MAN 6 - BRAND 1', 'MAN 3 - BRAND 1', 'MAN 5 - BRAND 2',
       'MAN 2 - BRAND 4', 'MAN 1 - OTHER BRANDS', 'MAN 5 - BRAND 1',
       'MAN 3 - BRAND 5', 'MAN 1 - BRAND 2', 'MAN 3 - BRAND 2'],
      dtype=object)

In [13]:
_df_ventas_historic['UNITS']=_df_ventas_historic['UNITS'].str.strip()
_df_ventas_historic['UNITS'].unique()

array(['1CT', '6CT', '12CT', '8CT', '10CT', '24CT', '4CT', '3CT', '18CT',
       '9CT', '28CT', '20CT', '5CT', '2CT', '16CT'], dtype=object)

In [14]:
_df_ventas_historic['VOLUME']=_df_ventas_historic['VOLUME'].str.strip()
_df_ventas_historic['VOLUME'].unique()

array(['330ML', '250ML', '300ML', '500ML', '355ML', '650ML', '1000ML',
       '375ML', '568ML', '440ML', '660ML', '200ML', '1100ML', '750ML',
       '1500ML', '210ML', '5000ML', '285ML', '450ML', '550ML', '473ML',
       '2000ML', '299ML', '735ML', '350ML', '377ML', '50000ML', '20000ML'],
      dtype=object)

In [15]:
_df_ventas_historic['factor_extrapolacion'].unique()

array(['11,48', '13,73', '15,36', ..., '26,22', '63,54', '112,71'],
      dtype=object)

In [16]:
_df_ventas_historic['cod_tienda'].unique()

array([243,  38,  30, 101, 113, 241, 214, 267,  59,  72, 283, 172,  68,
       126, 277, 236, 183, 143, 139,  97,  96, 271, 253, 227, 157, 147,
       111,  80,   8,  32, 249,  51,   6, 265, 174,  52,   3, 189, 185,
        56, 133,  31, 223,  61, 128,  15, 290, 288, 134, 255,  44,  36,
       149, 207, 121, 228, 182,  19, 144,   4, 280, 250,  55,  87, 164,
        57, 268, 124, 293, 242, 247,   5, 103, 117,  91,  20,  94, 148,
       171, 138, 274, 184, 125,  64, 300, 216, 213,  26, 205, 209, 262,
       235, 219,  67,  71, 108, 150,  39, 170,  82,  77, 263, 179, 291,
       142, 281, 106, 295,  88,  69,  54, 199, 146, 169,  14,  24,  74,
       270,  48, 114, 278, 230, 221, 272, 259, 297,  21,  50,  43, 192,
       194, 158, 226, 102, 229,  60, 233, 173, 225,  78, 136, 193, 264,
       276, 155, 107, 204, 137, 152, 252,  83,  12, 110, 285, 282, 246,
        35, 196,  42, 176, 238,  11, 248,  76, 239,  29, 296, 254,  47,
         7, 222,  66, 161, 195,  37, 119, 224,  70,  92, 260,  1

In [17]:
_df_ventas_historic['Canal'].unique()

array(['Supermercados', 'Hipermercados'], dtype=object)

In [18]:
_df_ventas_historic['Channel'].unique()

array(['Supermarkets', 'Hipermarkets'], dtype=object)

In [19]:
#month
month_mapping = {'October':10, 'May':5, 'September':9, 'July':7, 'December':12, 'April':4,'February':2, 'August':8, 'January':1, 'March':3, 'June':6, 'November':11}
#season
season_mapping = {'Fall':4, 'Spring':2, 'Summer':3, 'Winter':1}
#SEGMENT -> one-hot
#MANUFACTURER -> one-hot
#BRAND -> one-hot
#VOLUME
volume_mapping = {'330ML':6,'250ML':2,'300ML':5,'500ML':14,'355ML':8,'650ML':17,'1000ML':21,'375ML':9,'568ML':16,'440ML':11,'660ML':18,'200ML':0,'1100ML':22,'750ML':20,'1500ML':23,'210ML':1,'5000ML':27,'285ML':3,'450ML':12,'550ML':15,'473ML':13,'2000ML':24,'299ML':4,'735ML':19,'350ML':7,'377ML':10,'50000ML':27,'20000ML':26}
#UNITS
units_mapping = {'1CT':0,'6CT':5,'12CT':9,'8CT':6,'10CT':8,'24CT':13,'4CT':3,'3CT':2,'18CT':11,'9CT':7,'28CT':14,'20CT':12,'5CT':4,'2CT':1,'16CT':10}
#factor_extrapolacion
#Canal-> Discarded Channel in Spanish
#Channel -> Replace() Supermarkets -> 0 Hypermarkets -> 1 (Could be better one-hot)
#Provincia-Discarded
#Comunidad autónoma -> Discarded
#national_holidays_2021 ->Replace() 'N' -> 0 'Y' -> 1
#regional_holidays_2021->Replace() 'N' -> 0 'Y' -> 1
#local_holidays_2021->Replace() 'N' -> 0 'Y' -> 1
#national_holidays_2022->Replace() 'N' -> 0 'Y' -> 1
#regional_holidays_2022->Replace() 'N' -> 0 'Y' -> 1
#local_holidays_2022->Replace() 'N' -> 0 'Y' -> 1
#national_holidays_2023->Replace() 'N' -> 0 'Y' -> 1
#regional_holidays_2023->Replace() 'N' -> 0 'Y' -> 1
#local_holidays_2023->Replace() 'N' -> 0 'Y' -> 1



In [20]:
#columns to be one-hot encoded
one_hot_columns=['SEGMENT','MANUFACTURER','BRAND','PACKAGING']

In [21]:
# for loop to perform one-hot encoding using one_hot_columns
for column_name in one_hot_columns:
  onehot_column = pd.get_dummies(_df_ventas_historic[column_name],prefix=column_name)
  _df_ventas_historic_encoded = pd.get_dummies(_df_ventas_historic[column_name], prefix=column_name)
  _df_ventas_historic = pd.concat([_df_ventas_historic, _df_ventas_historic_encoded], axis=1).drop(column_name, axis=1)

In [22]:
_df_ventas_historic.head()

,Unnamed: 0,cod_tienda,cod_semana,cod_producto,ventas_unidades,ventas_valor,ventas_volumen,numero_referencias,precio_real_unidades,precio_real_volumen,precio_tarifa_unidades,precio_tarifa_volumen,month,year,season,CATEGORY,VOLUME,UNITS,promocion_cabecera,promocion_descuento,promocion_expositor,promocion_extra_cantidad,promocion_folleto,promocion_isla,promocion_multicompra,promocion_regalo,factor_extrapolacion,cod_canal,cod_provincia,postal_code,...,MANUFACTURER_MANUFACTURER 3,MANUFACTURER_MANUFACTURER 4,MANUFACTURER_MANUFACTURER 5,MANUFACTURER_MANUFACTURER 6,MANUFACTURER_OTHER MANUFACTURERS,BRAND_MAN 1 - BRAND 1,BRAND_MAN 1 - BRAND 2,BRAND_MAN 1 - OTHER BRANDS,BRAND_MAN 2 - BRAND 1,BRAND_MAN 2 - BRAND 2,BRAND_MAN 2 - BRAND 3,BRAND_MAN 2 - BRAND 4,BRAND_MAN 2 - BRAND 5,BRAND_MAN 2 - OTHER BRANDS,BRAND_MAN 3 - BRAND 1,BRAND_MAN 3 - BRAND 2,BRAND_MAN 3 - BRAND 3,BRAND_MAN 3 - BRAND 4,BRAND_MAN 3 - BRAND 5,BRAND_MAN 3 - OTHER BRANDS,BRAND_MAN 4 - BRAND 1,BRAND_MAN 5 - BRAND 1,BRAND_MAN 5 - BRAND 2,BRAND_MAN 5 - OTHER BRANDS,BRAND_MAN 6 - BRAND 1,BRAND_OTHER MAN - OTHER BRANDS,PACKAGING_CAN,PACKAGING_CRISTAL BOTTLE,PACKAGING_OTHER,PACKAGING_PET
0,0,243,42,236,30,19.50,10,1,0.65,0.33,0.65,0.33,October,2021,Fall,BEER,330ML,1CT,0,0,0,0,0,0,0,0,"11,48",1,26,26003,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,1,243,42,239,2,1.82,1,1,0.91,0.33,0.91,0.33,October,2021,Fall,BEER,330ML,1CT,0,0,0,0,0,0,0,0,"11,48",1,26,26003,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2,243,42,247,2,10.32,4,1,5.16,1.98,5.16,1.98,October,2021,Fall,BEER,330ML,6CT,0,0,0,0,0,0,0,0,"11,48",1,26,26003,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,3,243,42,258,3,14.97,6,1,4.99,1.98,4.99,1.98,October,2021,Fall,BEER,330ML,6CT,0,0,0,0,0,0,0,0,"11,48",1,26,26003,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
4,4,243,42,264,2,10.88,4,1,5.44,1.98,5.44,1.98,October,2021,Fall,BEER,330ML,6CT,0,0,0,0,0,0,0,0,"11,48",1,26,26003,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [23]:
#Ordinal encoding  for month
_df_ventas_historic['month']=_df_ventas_historic['month'].map(month_mapping)

In [24]:
#Ordinal encoding  for season
_df_ventas_historic['season']=_df_ventas_historic['season'].map(season_mapping)

In [25]:
#Ordinal encoding  for volume
_df_ventas_historic['VOLUME']=_df_ventas_historic['VOLUME'].map(volume_mapping)

In [26]:
#Ordinal encoding  for units
_df_ventas_historic['UNITS']=_df_ventas_historic['UNITS'].map(units_mapping)

In [27]:
replace_Y_N_columns=['national_holidays_2021','regional_holidays_2021','local_holidays_2021','national_holidays_2022','regional_holidays_2022','local_holidays_2022','national_holidays_2023','regional_holidays_2023','local_holidays_2023']
#national_holidays_2021 ->Replace() 'N' -> 0 'Y' -> 1
#regional_holidays_2021->Replace() 'N' -> 0 'Y' -> 1
#local_holidays_2021->Replace() 'N' -> 0 'Y' -> 1
#national_holidays_2022->Replace() 'N' -> 0 'Y' -> 1
#regional_holidays_2022->Replace() 'N' -> 0 'Y' -> 1
#local_holidays_2022->Replace() 'N' -> 0 'Y' -> 1
#national_holidays_2023->Replace() 'N' -> 0 'Y' -> 1
#regional_holidays_2023->Replace() 'N' -> 0 'Y' -> 1
#local_holidays_2023->Replace() 'N' -> 0 'Y' -> 1
for column_name in replace_Y_N_columns:
  _df_ventas_historic[column_name]=_df_ventas_historic[column_name].replace({'N':0,'Y':1})

In [28]:
#Remplazamos las comas por puntos
_df_ventas_historic['factor_extrapolacion']=_df_ventas_historic['factor_extrapolacion'].replace(',', '.', regex=True)

In [29]:
#convertimos a float
_df_ventas_historic['factor_extrapolacion']=_df_ventas_historic['factor_extrapolacion'].astype('float64')

In [30]:
#Creamos la columna ventas_valor_factor_extrapolacion que es el producto de ventas_valor por factor_extrapolacion
_df_ventas_historic['ventas_valor_factor_extrapolacion']=_df_ventas_historic['ventas_valor']*_df_ventas_historic['factor_extrapolacion']

In [31]:
_df_ventas_historic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5128394 entries, 0 to 5128393
Data columns (total 93 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   Unnamed: 0                         int64  
 1   cod_tienda                         int64  
 2   cod_semana                         int64  
 3   cod_producto                       int64  
 4   ventas_unidades                    int64  
 5   ventas_valor                       float64
 6   ventas_volumen                     int64  
 7   numero_referencias                 int64  
 8   precio_real_unidades               float64
 9   precio_real_volumen                float64
 10  precio_tarifa_unidades             float64
 11  precio_tarifa_volumen              float64
 12  month                              int64  
 13  year                               int64  
 14  season                             int64  
 15  CATEGORY                           object 
 16  VOLUME            

In [32]:
####aqui ya se convierte en cp4 --------------------- TODO COMPLETADO DE cp4
_df_ventas_historic_cp4=_df_ventas_historic.copy()

In [33]:
#################################################################################DESDE AQUO EJECUITo

In [34]:
#Remplazamos las comas por puntos
_df_ventas_historic_cp2['factor_extrapolacion']=_df_ventas_historic_cp2['factor_extrapolacion'].replace(',', '.', regex=True)

In [35]:
#convertimos a float
_df_ventas_historic_cp2['factor_extrapolacion']=_df_ventas_historic_cp2['factor_extrapolacion'].astype('float64')

In [36]:
#Creamos la columna ventas_valor_factor_extrapolacion que es el producto de ventas_valor por factor_extrapolacion
_df_ventas_historic_cp2['ventas_valor_factor_extrapolacion']=_df_ventas_historic_cp2['ventas_valor']*_df_ventas_historic_cp2['factor_extrapolacion']

In [38]:

sum_ventas_valor_factor_extrapolacion=_df_ventas_historic_cp2.groupby(['cod_tienda'])['ventas_valor_factor_extrapolacion'].sum().reset_index(name='ventas_valor_factor_extrapolacion_per_store')
sum_ventas_valor_factor_extrapolacion

,cod_tienda,ventas_valor_factor_extrapolacion_per_store
0,1,1.465832e+06
1,2,8.159604e+02
2,3,1.270568e+07
3,4,1.411304e+06
4,5,3.131477e+06
...,...,...
538,553,1.554187e+07
539,554,8.538197e+06
540,555,2.454924e+06
541,556,7.010913e+06


In [39]:
sum_ventas_valor_SEGMENT=_df_ventas_historic_cp2.groupby(['cod_tienda','SEGMENT'])['ventas_valor_factor_extrapolacion'].sum().reset_index(name='ventas_valor_brand_per_store')
sum_ventas_valor_SEGMENT

,cod_tienda,SEGMENT,ventas_valor_brand_per_store
0,1,BLACK,1.414698e+03
1,1,CELIACS,4.216480e+02
2,1,EXTRA,7.561654e+04
3,1,FLAVOURED,2.750632e+04
4,1,IMPORT PREMIUM,5.310461e+05
...,...,...,...
4699,557,IMPORT PREMIUM,5.727420e+05
4700,557,IMPORT SPECIAL,4.657594e+04
4701,557,NATIONAL,1.743152e+06
4702,557,WITHOUT ALCOHOL,8.050268e+04


In [40]:

sum_ventas_valor_SEGMENT['SEGMENT']="SEGMENT_"+sum_ventas_valor_SEGMENT['SEGMENT'].astype(str)
sum_ventas_valor_SEGMENT

,cod_tienda,SEGMENT,ventas_valor_brand_per_store
0,1,SEGMENT_BLACK,1.414698e+03
1,1,SEGMENT_CELIACS,4.216480e+02
2,1,SEGMENT_EXTRA,7.561654e+04
3,1,SEGMENT_FLAVOURED,2.750632e+04
4,1,SEGMENT_IMPORT PREMIUM,5.310461e+05
...,...,...,...
4699,557,SEGMENT_IMPORT PREMIUM,5.727420e+05
4700,557,SEGMENT_IMPORT SPECIAL,4.657594e+04
4701,557,SEGMENT_NATIONAL,1.743152e+06
4702,557,SEGMENT_WITHOUT ALCOHOL,8.050268e+04


In [41]:
sum_ventas_valor_SEGMENT_pivot = sum_ventas_valor_SEGMENT.pivot(index='cod_tienda', columns='SEGMENT', values='ventas_valor_brand_per_store')
sum_ventas_valor_SEGMENT_pivot

SEGMENT,SEGMENT_ARTISANS,SEGMENT_BLACK,SEGMENT_CELIACS,SEGMENT_EXTRA,SEGMENT_FLAVOURED,SEGMENT_IMPORT PREMIUM,SEGMENT_IMPORT SPECIAL,SEGMENT_NATIONAL,SEGMENT_WITHOUT ALCOHOL,SEGMENT_ZERO ALCOHOL
cod_tienda,,,,,,,,,,
1,NaN,1414.6977,421.6480,7.561654e+04,27506.3214,5.310461e+05,3305.1734,6.300099e+05,71392.9384,125118.5008
2,NaN,NaN,NaN,8.159604e+02,NaN,NaN,NaN,NaN,NaN,NaN
3,181071.9192,126202.5774,62344.3673,2.056731e+06,678890.3149,1.315005e+06,391735.2451,7.045113e+06,369141.4429,479443.2445
4,22195.1685,48411.6924,4216.3742,2.091155e+05,77684.2095,2.629581e+05,150471.2403,5.378297e+05,40797.3675,57624.5116
5,NaN,NaN,2804.5336,1.216046e+05,79013.7463,6.986261e+05,41703.5397,1.931208e+06,143889.3444,112627.8304
...,...,...,...,...,...,...,...,...,...,...
553,34137.5301,64338.3038,44759.6194,7.102494e+05,506361.3887,4.687378e+06,275582.2222,8.084835e+06,554765.9192,579462.9100
554,NaN,10115.7618,25409.1593,3.395601e+05,143621.9785,2.820922e+06,95464.0766,4.283443e+06,475089.6387,344571.2952
555,10943.7078,38389.4371,25615.2839,2.736880e+05,119030.5242,2.085809e+05,90751.9777,1.519692e+06,1599.3954,166633.4643


In [42]:
sum_ventas_valor_brand=_df_ventas_historic_cp2.groupby(['cod_tienda','BRAND'])['ventas_valor_factor_extrapolacion'].sum().reset_index(name='ventas_valor_brand_per_store')
sum_ventas_valor_brand

,cod_tienda,BRAND,ventas_valor_brand_per_store
0,1,MAN 1 - BRAND 1,20931.8845
1,1,MAN 2 - BRAND 1,31273.6272
2,1,MAN 2 - BRAND 2,245808.4237
3,1,MAN 2 - BRAND 3,34296.4296
4,1,MAN 2 - BRAND 4,381.2744
...,...,...,...
7976,557,MAN 3 - BRAND 4,457165.1613
7977,557,MAN 3 - OTHER BRANDS,32954.9760
7978,557,MAN 5 - BRAND 2,19806.8153
7979,557,MAN 6 - BRAND 1,589364.2085


In [43]:
sum_ventas_valor_brand_pivot = sum_ventas_valor_brand.pivot(index='cod_tienda', columns='BRAND', values='ventas_valor_brand_per_store')
sum_ventas_valor_brand_pivot

BRAND,MAN 1 - BRAND 1,MAN 1 - BRAND 2,MAN 1 - OTHER BRANDS,MAN 2 - BRAND 1,MAN 2 - BRAND 2,MAN 2 - BRAND 3,MAN 2 - BRAND 4,MAN 2 - BRAND 5,MAN 2 - OTHER BRANDS,MAN 3 - BRAND 1,MAN 3 - BRAND 2,MAN 3 - BRAND 3,MAN 3 - BRAND 4,MAN 3 - BRAND 5,MAN 3 - OTHER BRANDS,MAN 4 - BRAND 1,MAN 5 - BRAND 1,MAN 5 - BRAND 2,MAN 5 - OTHER BRANDS,MAN 6 - BRAND 1,OTHER MAN - OTHER BRANDS
cod_tienda,,,,,,,,,,,,,,,,,,,,,
1,2.093188e+04,NaN,NaN,3.127363e+04,245808.4237,34296.4296,381.2744,2789.4838,78053.2296,9.120631e+04,NaN,7.970274e+04,1.489422e+05,NaN,14577.3146,NaN,NaN,1213.6000,NaN,7.044279e+05,12227.5290
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.159604e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.027358e+06,NaN,132435.0502,1.270758e+06,406712.9309,896062.5975,38137.5001,79546.7018,364070.1169,6.454752e+05,NaN,3.162244e+06,1.942609e+06,NaN,102719.8487,1.586968e+06,82.2500,90054.6162,NaN,1.083251e+05,852118.1660
4,1.327195e+05,17.0250,8789.1847,9.422291e+04,34089.0494,141834.0120,11071.3784,15241.1026,41484.0975,5.562945e+04,832.7146,1.199536e+05,2.091851e+05,NaN,28714.5962,1.638660e+05,695.7223,12907.7715,51.7644,5.221909e+04,287779.6831
5,8.297298e+04,NaN,2650.3022,5.530031e+05,65193.1952,94031.3046,NaN,41703.5397,64664.9017,1.145855e+05,NaN,1.315183e+05,5.700949e+05,NaN,NaN,7.320569e+04,NaN,2804.5336,NaN,1.318352e+06,16696.9154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,2.705096e+05,51714.9963,100219.1811,2.793034e+06,363669.7229,765827.1698,15288.9601,77586.9689,238446.1472,1.234292e+06,1459.1147,3.424622e+06,1.043402e+06,NaN,299899.9401,4.280039e+05,82.4850,78140.7906,2751.5047,4.019239e+06,333681.4539
554,1.457682e+05,147459.1511,NaN,2.141617e+06,218040.2488,628907.9155,2764.2759,46065.5712,183230.7452,8.065940e+05,NaN,9.688083e+05,4.404443e+05,NaN,110550.7681,3.747018e+05,NaN,NaN,NaN,2.226328e+06,96917.2022
555,1.303386e+06,NaN,122706.8132,1.211665e+04,22975.2967,120953.9098,13399.1583,18796.7054,18780.2744,2.743587e+04,NaN,4.218047e+03,2.658553e+05,NaN,38153.8431,NaN,194348.1951,NaN,NaN,1.704327e+05,121365.5460


In [44]:
provinces=_df_ventas_historic_cp2.groupby('cod_tienda')['Provincia'].unique().explode().reset_index(name='Provincia')
provinces.head()

,cod_tienda,Provincia
0,1,Almeria
1,2,Area Metropolitana de Madrid
2,3,Cantabria
3,4,Vizcaya
4,5,Castellon


In [45]:
_df_ventas_historic_cp4

,Unnamed: 0,cod_tienda,cod_semana,cod_producto,ventas_unidades,ventas_valor,ventas_volumen,numero_referencias,precio_real_unidades,precio_real_volumen,precio_tarifa_unidades,precio_tarifa_volumen,month,year,season,CATEGORY,VOLUME,UNITS,promocion_cabecera,promocion_descuento,promocion_expositor,promocion_extra_cantidad,promocion_folleto,promocion_isla,promocion_multicompra,promocion_regalo,factor_extrapolacion,cod_canal,cod_provincia,postal_code,...,MANUFACTURER_MANUFACTURER 4,MANUFACTURER_MANUFACTURER 5,MANUFACTURER_MANUFACTURER 6,MANUFACTURER_OTHER MANUFACTURERS,BRAND_MAN 1 - BRAND 1,BRAND_MAN 1 - BRAND 2,BRAND_MAN 1 - OTHER BRANDS,BRAND_MAN 2 - BRAND 1,BRAND_MAN 2 - BRAND 2,BRAND_MAN 2 - BRAND 3,BRAND_MAN 2 - BRAND 4,BRAND_MAN 2 - BRAND 5,BRAND_MAN 2 - OTHER BRANDS,BRAND_MAN 3 - BRAND 1,BRAND_MAN 3 - BRAND 2,BRAND_MAN 3 - BRAND 3,BRAND_MAN 3 - BRAND 4,BRAND_MAN 3 - BRAND 5,BRAND_MAN 3 - OTHER BRANDS,BRAND_MAN 4 - BRAND 1,BRAND_MAN 5 - BRAND 1,BRAND_MAN 5 - BRAND 2,BRAND_MAN 5 - OTHER BRANDS,BRAND_MAN 6 - BRAND 1,BRAND_OTHER MAN - OTHER BRANDS,PACKAGING_CAN,PACKAGING_CRISTAL BOTTLE,PACKAGING_OTHER,PACKAGING_PET,ventas_valor_factor_extrapolacion
0,0,243,42,236,30,19.50,10,1,0.65,0.33,0.65,0.33,10,2021,4,BEER,6,0,0,0,0,0,0,0,0,0,11.48,1,26,26003,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,223.8600
1,1,243,42,239,2,1.82,1,1,0.91,0.33,0.91,0.33,10,2021,4,BEER,6,0,0,0,0,0,0,0,0,0,11.48,1,26,26003,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,20.8936
2,2,243,42,247,2,10.32,4,1,5.16,1.98,5.16,1.98,10,2021,4,BEER,6,5,0,0,0,0,0,0,0,0,11.48,1,26,26003,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,118.4736
3,3,243,42,258,3,14.97,6,1,4.99,1.98,4.99,1.98,10,2021,4,BEER,6,5,0,0,0,0,0,0,0,0,11.48,1,26,26003,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,171.8556
4,4,243,42,264,2,10.88,4,1,5.44,1.98,5.44,1.98,10,2021,4,BEER,6,5,0,0,0,0,0,0,0,0,11.48,1,26,26003,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,124.9024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5128389,5128389,526,47,625,100,127.00,36,1,1.27,0.36,1.27,0.36,11,2021,4,BEER,8,0,0,0,0,0,0,0,0,0,1.89,2,91,8029,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,240.0300
5128390,5128390,526,47,633,20,49.00,10,1,2.45,0.50,2.45,0.50,11,2021,4,BEER,14,0,0,0,0,0,0,0,0,0,1.89,2,91,8029,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,92.6100
5128391,5128391,526,47,652,6,37.50,18,1,6.25,3.00,6.25,3.00,11,2021,4,BEER,2,9,0,0,0,0,0,0,0,0,1.89,2,91,8029,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,70.8750
5128392,5128392,526,47,654,23,85.10,35,1,3.70,1.50,3.70,1.50,11,2021,4,BEER,2,5,0,0,0,0,0,0,0,0,1.89,2,91,8029,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,160.8390


In [46]:
df_ventas_historic_cluster_final=_df_ventas_historic_cp4.groupby('cod_tienda').agg(lambda x: int((x == 1).any()))

df_ventas_historic_cluster_final

,Unnamed: 0,cod_semana,cod_producto,ventas_unidades,ventas_valor,ventas_volumen,numero_referencias,precio_real_unidades,precio_real_volumen,precio_tarifa_unidades,precio_tarifa_volumen,month,year,season,CATEGORY,VOLUME,UNITS,promocion_cabecera,promocion_descuento,promocion_expositor,promocion_extra_cantidad,promocion_folleto,promocion_isla,promocion_multicompra,promocion_regalo,factor_extrapolacion,cod_canal,cod_provincia,postal_code,sales_surface_sqmeters,...,MANUFACTURER_MANUFACTURER 4,MANUFACTURER_MANUFACTURER 5,MANUFACTURER_MANUFACTURER 6,MANUFACTURER_OTHER MANUFACTURERS,BRAND_MAN 1 - BRAND 1,BRAND_MAN 1 - BRAND 2,BRAND_MAN 1 - OTHER BRANDS,BRAND_MAN 2 - BRAND 1,BRAND_MAN 2 - BRAND 2,BRAND_MAN 2 - BRAND 3,BRAND_MAN 2 - BRAND 4,BRAND_MAN 2 - BRAND 5,BRAND_MAN 2 - OTHER BRANDS,BRAND_MAN 3 - BRAND 1,BRAND_MAN 3 - BRAND 2,BRAND_MAN 3 - BRAND 3,BRAND_MAN 3 - BRAND 4,BRAND_MAN 3 - BRAND 5,BRAND_MAN 3 - OTHER BRANDS,BRAND_MAN 4 - BRAND 1,BRAND_MAN 5 - BRAND 1,BRAND_MAN 5 - BRAND 2,BRAND_MAN 5 - OTHER BRANDS,BRAND_MAN 6 - BRAND 1,BRAND_OTHER MAN - OTHER BRANDS,PACKAGING_CAN,PACKAGING_CRISTAL BOTTLE,PACKAGING_OTHER,PACKAGING_PET,ventas_valor_factor_extrapolacion
cod_tienda,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,1,1,1,1,1,1,0,1,1,0,1,0,0,0,1,1,1,1,0,1,1,1,0,1,0,0,0,...,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0,1,1,0,1,0,0,1,0,1,1,1,1,0,1,0
2,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,1,0,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,1,1,1,1,1,1,1,0,1,0,0,0,...,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,0,1,1,1,1,1,1,0
4,0,1,0,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0
5,0,1,0,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,1,1,1,1,1,0,1,0,1,0,0,0,...,1,1,1,1,1,0,1,1,1,1,0,1,1,1,0,1,1,0,0,1,0,1,0,1,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,0,1,0,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0
554,0,1,0,1,1,1,1,1,1,1,1,1,0,1,0,0,0,1,1,1,1,1,1,1,1,0,1,0,0,0,...,1,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,1,1,0,0,0,1,1,1,1,0,1,0
555,0,1,0,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,1,1,0,1,1,0,1,0,1,0,0,0,...,0,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,0


In [47]:
df_ventas_historic_cluster_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 543 entries, 1 to 557
Data columns (total 92 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Unnamed: 0                         543 non-null    int64
 1   cod_semana                         543 non-null    int64
 2   cod_producto                       543 non-null    int64
 3   ventas_unidades                    543 non-null    int64
 4   ventas_valor                       543 non-null    int64
 5   ventas_volumen                     543 non-null    int64
 6   numero_referencias                 543 non-null    int64
 7   precio_real_unidades               543 non-null    int64
 8   precio_real_volumen                543 non-null    int64
 9   precio_tarifa_unidades             543 non-null    int64
 10  precio_tarifa_volumen              543 non-null    int64
 11  month                              543 non-null    int64
 12  year                  

In [48]:
# Drop columns 
df_ventas_historic_cluster_final.drop(columns=['Unnamed: 0','cod_semana','cod_producto','ventas_unidades','ventas_valor','ventas_volumen','numero_referencias','precio_real_unidades','precio_real_volumen','precio_tarifa_unidades','precio_tarifa_volumen','month','year','season','CATEGORY','VOLUME','UNITS','cod_provincia','postal_code','sales_surface_sqmeters','Canal','Channel','Provincia','Comunidad autónoma','Lat','Lon','TEMP_MINIMA','TEMP_MAXIMA','TEMP_MEDIA','PRECIPITACION','ventas_valor_factor_extrapolacion'],inplace=True)
df_ventas_historic_cluster_final

,promocion_cabecera,promocion_descuento,promocion_expositor,promocion_extra_cantidad,promocion_folleto,promocion_isla,promocion_multicompra,promocion_regalo,factor_extrapolacion,cod_canal,national_holidays_2021,regional_holidays_2021,local_holidays_2021,national_holidays_2022,regional_holidays_2022,local_holidays_2022,national_holidays_2023,regional_holidays_2023,local_holidays_2023,SEGMENT_ARTISANS,SEGMENT_BLACK,SEGMENT_CELIACS,SEGMENT_EXTRA,SEGMENT_FLAVOURED,SEGMENT_IMPORT PREMIUM,SEGMENT_IMPORT SPECIAL,SEGMENT_NATIONAL,SEGMENT_WITHOUT ALCOHOL,SEGMENT_ZERO ALCOHOL,MANUFACTURER_MANUFACTURER 1,...,MANUFACTURER_MANUFACTURER 3,MANUFACTURER_MANUFACTURER 4,MANUFACTURER_MANUFACTURER 5,MANUFACTURER_MANUFACTURER 6,MANUFACTURER_OTHER MANUFACTURERS,BRAND_MAN 1 - BRAND 1,BRAND_MAN 1 - BRAND 2,BRAND_MAN 1 - OTHER BRANDS,BRAND_MAN 2 - BRAND 1,BRAND_MAN 2 - BRAND 2,BRAND_MAN 2 - BRAND 3,BRAND_MAN 2 - BRAND 4,BRAND_MAN 2 - BRAND 5,BRAND_MAN 2 - OTHER BRANDS,BRAND_MAN 3 - BRAND 1,BRAND_MAN 3 - BRAND 2,BRAND_MAN 3 - BRAND 3,BRAND_MAN 3 - BRAND 4,BRAND_MAN 3 - BRAND 5,BRAND_MAN 3 - OTHER BRANDS,BRAND_MAN 4 - BRAND 1,BRAND_MAN 5 - BRAND 1,BRAND_MAN 5 - BRAND 2,BRAND_MAN 5 - OTHER BRANDS,BRAND_MAN 6 - BRAND 1,BRAND_OTHER MAN - OTHER BRANDS,PACKAGING_CAN,PACKAGING_CRISTAL BOTTLE,PACKAGING_OTHER,PACKAGING_PET
cod_tienda,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0,1,1,0,1,0,0,1,0,1,1,1,1,0,1
2,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,0,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1
5,0,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,0,1,1,0,0,1,0,1,0,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1
554,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,1,1,0,0,0,1,1,1,1,0,1
555,0,1,1,0,1,1,0,1,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0


In [49]:
df_ventas_historic_cluster_final_merged = pd.merge(df_ventas_historic_cluster_final, sum_ventas_valor_brand_pivot, on='cod_tienda')
df_ventas_historic_cluster_final_merged

,promocion_cabecera,promocion_descuento,promocion_expositor,promocion_extra_cantidad,promocion_folleto,promocion_isla,promocion_multicompra,promocion_regalo,factor_extrapolacion,cod_canal,national_holidays_2021,regional_holidays_2021,local_holidays_2021,national_holidays_2022,regional_holidays_2022,local_holidays_2022,national_holidays_2023,regional_holidays_2023,local_holidays_2023,SEGMENT_ARTISANS,SEGMENT_BLACK,SEGMENT_CELIACS,SEGMENT_EXTRA,SEGMENT_FLAVOURED,SEGMENT_IMPORT PREMIUM,SEGMENT_IMPORT SPECIAL,SEGMENT_NATIONAL,SEGMENT_WITHOUT ALCOHOL,SEGMENT_ZERO ALCOHOL,MANUFACTURER_MANUFACTURER 1,...,BRAND_MAN 5 - BRAND 1,BRAND_MAN 5 - BRAND 2,BRAND_MAN 5 - OTHER BRANDS,BRAND_MAN 6 - BRAND 1,BRAND_OTHER MAN - OTHER BRANDS,PACKAGING_CAN,PACKAGING_CRISTAL BOTTLE,PACKAGING_OTHER,PACKAGING_PET,MAN 1 - BRAND 1,MAN 1 - BRAND 2,MAN 1 - OTHER BRANDS,MAN 2 - BRAND 1,MAN 2 - BRAND 2,MAN 2 - BRAND 3,MAN 2 - BRAND 4,MAN 2 - BRAND 5,MAN 2 - OTHER BRANDS,MAN 3 - BRAND 1,MAN 3 - BRAND 2,MAN 3 - BRAND 3,MAN 3 - BRAND 4,MAN 3 - BRAND 5,MAN 3 - OTHER BRANDS,MAN 4 - BRAND 1,MAN 5 - BRAND 1,MAN 5 - BRAND 2,MAN 5 - OTHER BRANDS,MAN 6 - BRAND 1,OTHER MAN - OTHER BRANDS
cod_tienda,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,...,0,1,0,1,1,1,1,0,1,2.093188e+04,NaN,NaN,3.127363e+04,245808.4237,34296.4296,381.2744,2789.4838,78053.2296,9.120631e+04,NaN,7.970274e+04,1.489422e+05,NaN,14577.3146,NaN,NaN,1213.6000,NaN,7.044279e+05,12227.5290
2,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.159604e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1.027358e+06,NaN,132435.0502,1.270758e+06,406712.9309,896062.5975,38137.5001,79546.7018,364070.1169,6.454752e+05,NaN,3.162244e+06,1.942609e+06,NaN,102719.8487,1.586968e+06,82.2500,90054.6162,NaN,1.083251e+05,852118.1660
4,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1.327195e+05,17.0250,8789.1847,9.422291e+04,34089.0494,141834.0120,11071.3784,15241.1026,41484.0975,5.562945e+04,832.7146,1.199536e+05,2.091851e+05,NaN,28714.5962,1.638660e+05,695.7223,12907.7715,51.7644,5.221909e+04,287779.6831
5,0,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,...,0,1,0,1,1,1,1,0,1,8.297298e+04,NaN,2650.3022,5.530031e+05,65193.1952,94031.3046,NaN,41703.5397,64664.9017,1.145855e+05,NaN,1.315183e+05,5.700949e+05,NaN,NaN,7.320569e+04,NaN,2804.5336,NaN,1.318352e+06,16696.9154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,2.705096e+05,51714.9963,100219.1811,2.793034e+06,363669.7229,765827.1698,15288.9601,77586.9689,238446.1472,1.234292e+06,1459.1147,3.424622e+06,1.043402e+06,NaN,299899.9401,4.280039e+05,82.4850,78140.7906,2751.5047,4.019239e+06,333681.4539
554,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,1,1,1,1,0,1,1.457682e+05,147459.1511,NaN,2.141617e+06,218040.2488,628907.9155,2764.2759,46065.5712,183230.7452,8.065940e+05,NaN,9.688083e+05,4.404443e+05,NaN,110550.7681,3.747018e+05,NaN,NaN,NaN,2.226328e+06,96917.2022
555,0,1,1,0,1,1,0,1,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,...,1,0,0,1,1,1,1,1,0,1.303386e+06,NaN,122706.8132,1.211665e+04,22975.2967,120953.9098,13399.1583,18796.7054,18780.2744,2.743587e+04,NaN,4.218047e+03,2.658553e+05,NaN,38153.8431,NaN,194348.1951,NaN,NaN,1.704327e+05,121365.5460


In [50]:
df_ventas_historic_cluster_final_merged = pd.merge(df_ventas_historic_cluster_final_merged, sum_ventas_valor_SEGMENT_pivot, on='cod_tienda')
df_ventas_historic_cluster_final_merged

,promocion_cabecera,promocion_descuento,promocion_expositor,promocion_extra_cantidad,promocion_folleto,promocion_isla,promocion_multicompra,promocion_regalo,factor_extrapolacion,cod_canal,national_holidays_2021,regional_holidays_2021,local_holidays_2021,national_holidays_2022,regional_holidays_2022,local_holidays_2022,national_holidays_2023,regional_holidays_2023,local_holidays_2023,SEGMENT_ARTISANS_x,SEGMENT_BLACK_x,SEGMENT_CELIACS_x,SEGMENT_EXTRA_x,SEGMENT_FLAVOURED_x,SEGMENT_IMPORT PREMIUM_x,SEGMENT_IMPORT SPECIAL_x,SEGMENT_NATIONAL_x,SEGMENT_WITHOUT ALCOHOL_x,SEGMENT_ZERO ALCOHOL_x,MANUFACTURER_MANUFACTURER 1,...,MAN 1 - BRAND 2,MAN 1 - OTHER BRANDS,MAN 2 - BRAND 1,MAN 2 - BRAND 2,MAN 2 - BRAND 3,MAN 2 - BRAND 4,MAN 2 - BRAND 5,MAN 2 - OTHER BRANDS,MAN 3 - BRAND 1,MAN 3 - BRAND 2,MAN 3 - BRAND 3,MAN 3 - BRAND 4,MAN 3 - BRAND 5,MAN 3 - OTHER BRANDS,MAN 4 - BRAND 1,MAN 5 - BRAND 1,MAN 5 - BRAND 2,MAN 5 - OTHER BRANDS,MAN 6 - BRAND 1,OTHER MAN - OTHER BRANDS,SEGMENT_ARTISANS_y,SEGMENT_BLACK_y,SEGMENT_CELIACS_y,SEGMENT_EXTRA_y,SEGMENT_FLAVOURED_y,SEGMENT_IMPORT PREMIUM_y,SEGMENT_IMPORT SPECIAL_y,SEGMENT_NATIONAL_y,SEGMENT_WITHOUT ALCOHOL_y,SEGMENT_ZERO ALCOHOL_y
cod_tienda,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,...,NaN,NaN,3.127363e+04,245808.4237,34296.4296,381.2744,2789.4838,78053.2296,9.120631e+04,NaN,7.970274e+04,1.489422e+05,NaN,14577.3146,NaN,NaN,1213.6000,NaN,7.044279e+05,12227.5290,NaN,1414.6977,421.6480,7.561654e+04,27506.3214,5.310461e+05,3305.1734,6.300099e+05,71392.9384,125118.5008
2,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.159604e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.159604e+02,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,NaN,132435.0502,1.270758e+06,406712.9309,896062.5975,38137.5001,79546.7018,364070.1169,6.454752e+05,NaN,3.162244e+06,1.942609e+06,NaN,102719.8487,1.586968e+06,82.2500,90054.6162,NaN,1.083251e+05,852118.1660,181071.9192,126202.5774,62344.3673,2.056731e+06,678890.3149,1.315005e+06,391735.2451,7.045113e+06,369141.4429,479443.2445
4,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,17.0250,8789.1847,9.422291e+04,34089.0494,141834.0120,11071.3784,15241.1026,41484.0975,5.562945e+04,832.7146,1.199536e+05,2.091851e+05,NaN,28714.5962,1.638660e+05,695.7223,12907.7715,51.7644,5.221909e+04,287779.6831,22195.1685,48411.6924,4216.3742,2.091155e+05,77684.2095,2.629581e+05,150471.2403,5.378297e+05,40797.3675,57624.5116
5,0,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,...,NaN,2650.3022,5.530031e+05,65193.1952,94031.3046,NaN,41703.5397,64664.9017,1.145855e+05,NaN,1.315183e+05,5.700949e+05,NaN,NaN,7.320569e+04,NaN,2804.5336,NaN,1.318352e+06,16696.9154,NaN,NaN,2804.5336,1.216046e+05,79013.7463,6.986261e+05,41703.5397,1.931208e+06,143889.3444,112627.8304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,51714.9963,100219.1811,2.793034e+06,363669.7229,765827.1698,15288.9601,77586.9689,238446.1472,1.234292e+06,1459.1147,3.424622e+06,1.043402e+06,NaN,299899.9401,4.280039e+05,82.4850,78140.7906,2751.5047,4.019239e+06,333681.4539,34137.5301,64338.3038,44759.6194,7.102494e+05,506361.3887,4.687378e+06,275582.2222,8.084835e+06,554765.9192,579462.9100
554,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,...,147459.1511,NaN,2.141617e+06,218040.2488,628907.9155,2764.2759,46065.5712,183230.7452,8.065940e+05,NaN,9.688083e+05,4.404443e+05,NaN,110550.7681,3.747018e+05,NaN,NaN,NaN,2.226328e+06,96917.2022,NaN,10115.7618,25409.1593,3.395601e+05,143621.9785,2.820922e+06,95464.0766,4.283443e+06,475089.6387,344571.2952
555,0,1,1,0,1,

In [61]:
###### Calculo ventas valor por tienda
sum_ventas_valor=_df_ventas_historic_cp2.groupby(['cod_tienda'])['ventas_valor'].sum().reset_index(name='ventas_valor_per_store')


#sum_ventas_valor=_df_ventas_historic.groupby(['cod_tienda']).apply(lambda x: x['ventas_valor'].sum()/ x['sales_surface_sqmeters']).reset_index(name='ventas_valor_sqmeter')

_df_ventas_historic_cp2=_df_ventas_historic_cp2.merge(sum_ventas_valor, on='cod_tienda', how='left')

In [58]:
_df_ventas_historic_cp2['Size_store_sq_meters']=_df_ventas_historic_cp2['sales_surface_sqmeters'].apply(map_store_sq_meters)

In [59]:
_df_ventas_historic_cp2


,Unnamed: 0,cod_tienda,cod_semana,cod_producto,ventas_unidades,ventas_valor,ventas_volumen,numero_referencias,precio_real_unidades,precio_real_volumen,precio_tarifa_unidades,precio_tarifa_volumen,month,year,season,CATEGORY,SEGMENT,MANUFACTURER,BRAND,PACKAGING,VOLUME,UNITS,promocion_cabecera,promocion_descuento,promocion_expositor,promocion_extra_cantidad,promocion_folleto,promocion_isla,promocion_multicompra,promocion_regalo,factor_extrapolacion,cod_canal,cod_provincia,postal_code,sales_surface_sqmeters,Canal,Channel,Provincia,Comunidad autónoma,Lat,Lon,TEMP_MINIMA,TEMP_MAXIMA,TEMP_MEDIA,PRECIPITACION,national_holidays_2021,regional_holidays_2021,local_holidays_2021,national_holidays_2022,regional_holidays_2022,local_holidays_2022,national_holidays_2023,regional_holidays_2023,local_holidays_2023,ventas_valor_factor_extrapolacion,Size_store_sq_meters
0,0,243,42,236,30,19.50,10,1,0.65,0.33,0.65,0.33,October,2021,Fall,BEER,EXTRA,MANUFACTURER 2,MAN 2 - BRAND 1,CAN,330ML,1CT,0,0,0,0,0,0,0,0,11.48,1,26,26003,1500,Supermercados,Supermarkets,La Rioja,La Rioja,42.27474,-2.51708,5.671429,12.285714,8.800000,1.285714,N,N,N,N,N,N,N,N,N,223.8600,3
1,1,243,42,239,2,1.82,1,1,0.91,0.33,0.91,0.33,October,2021,Fall,BEER,EXTRA,MANUFACTURER 2,MAN 2 - BRAND 1,CRISTAL BOTTLE,330ML,1CT,0,0,0,0,0,0,0,0,11.48,1,26,26003,1500,Supermercados,Supermarkets,La Rioja,La Rioja,42.27474,-2.51708,5.671429,12.285714,8.800000,1.285714,N,N,N,N,N,N,N,N,N,20.8936,3
2,2,243,42,247,2,10.32,4,1,5.16,1.98,5.16,1.98,October,2021,Fall,BEER,EXTRA,MANUFACTURER 2,MAN 2 - BRAND 2,CRISTAL BOTTLE,330ML,6CT,0,0,0,0,0,0,0,0,11.48,1,26,26003,1500,Supermercados,Supermarkets,La Rioja,La Rioja,42.27474,-2.51708,5.671429,12.285714,8.800000,1.285714,N,N,N,N,N,N,N,N,N,118.4736,3
3,3,243,42,258,3,14.97,6,1,4.99,1.98,4.99,1.98,October,2021,Fall,BEER,EXTRA,MANUFACTURER 4,MAN 4 - BRAND 1,CRISTAL BOTTLE,330ML,6CT,0,0,0,0,0,0,0,0,11.48,1,26,26003,1500,Supermercados,Supermarkets,La Rioja,La Rioja,42.27474,-2.51708,5.671429,12.285714,8.800000,1.285714,N,N,N,N,N,N,N,N,N,171.8556,3
4,4,243,42,264,2,10.88,4,1,5.44,1.98,5.44,1.98,October,2021,Fall,BEER,EXTRA,MANUFACTURER 4,MAN 4 - BRAND 1,CRISTAL BOTTLE,330ML,6CT,0,0,0,0,0,0,0,0,11.48,1,26,26003,1500,Supermercados,Supermarkets,La Rioja,La Rioja,42.27474,-2.51708,5.671429,12.285714,8.800000,1.285714,N,N,N,N,N,N,N,N,N,124.9024,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5128389,5128389,526,47,625,100,127.00,36,1,1.27,0.36,1.27,0.36,November,2021,Fall,BEER,IMPORT PREMIUM,OTHER MANUFACTURERS,OTHER MAN - OTHER BRANDS,CRISTAL BOTTLE,355ML,1CT,0,0,0,0,0,0,0,0,1.89,2,91,8029,4500,Hipermercados,Hipermarkets,Area Metropolitana de Barcelona,Cataluña,41.37591,2.14988,7.657143,14.114286,10.442857,1.242857,N,N,N,N,N,Y,N,N,N,240.0300,4
5128390,5128390,526,47,633,20,49.00,10,1,2.45,0.50,2.45,0.50,November,2021,Fall,BEER,IMPORT PREMIUM,OTHER MANUFACTURERS,OTHER MAN - OTHER BRANDS,CRISTAL BOTTLE,500ML,1CT,0,0,0,0,0,0,0,0,1.89,2,91,8029,4500,Hipermercados,Hipermarkets,Area Metropolitana de Barcelona,Cataluña,41.37591,2.14988,7.657143,14.114286,10.442857,1.242857,N,N,N,N,N,Y,N,N,N,92.6100,4
5128391,5128391,526,47,652,6,37.50,18,1,6.25,3.00,6.25,3.00,November,2021,Fall,BEER,IMPORT PREMIUM,MANUFACTURER 2,MAN 2 - BRAND 3,CRISTAL BOTTLE,250ML,12CT,0,0,0,0,0,0,0,0,1.89,2,91,8029,4500,Hipermercados,Hipermarkets,Area Metropolitana de Barcelona,Cataluña,41.37591,2.14988,7.657143,14.114286,10.442857,1.242857,N,N,N,N,N,Y,N,N,N,70.8750,4
5128392,5128392,526,47,654,23,85.10,35,1,3.70,1.50,3.70,1.50,November,2021,Fall,BEER,IMPORT PREMIUM,MANUFACTURER 2,MAN 2 - BRAND 3,CRISTAL BOTTLE,250ML,6CT,0,0,0,0,0,0,0,0,1.89,2,91,8029,4500,Hipermercados,Hipermarkets,Area Metropolitana de Barcelona,Cataluña,41.37591,2.14988,7.657143,14.114286,10.442857,1.242857,N,N,N,N,N,Y,N,N,N,160.8390,4


In [62]:
_df_ventas_historic_cluster_with_sq=_df_ventas_historic_cp2.drop_duplicates(subset='cod_tienda')[['cod_tienda','sales_surface_sqmeters','ventas_valor_per_store','Channel',]].reset_index(drop=True)

In [64]:
_df_ventas_historic_cluster_with_sq['revenue_per_sqmeter']=_df_ventas_historic_cluster_with_sq['ventas_valor_per_store']/_df_ventas_historic_cluster_with_sq['sales_surface_sqmeters']

In [65]:
_df_ventas_historic_cluster_with_sq


,cod_tienda,sales_surface_sqmeters,ventas_valor_per_store,Channel,revenue_per_sqmeter
0,243,1500,376962.96,Supermarkets,251.308640
1,38,600,283402.85,Supermarkets,472.338083
2,30,400,217143.08,Supermarkets,542.857700
3,101,1100,354598.14,Supermarkets,322.361945
4,113,600,191223.41,Supermarkets,318.705683
...,...,...,...,...,...
538,431,300,277.54,Supermarkets,0.925133
539,350,1300,19.68,Supermarkets,0.015138
540,494,200,317.99,Supermarkets,1.589950
541,552,600,531.13,Supermarkets,0.885217


In [66]:
df_ventas_historic_cluster_final_merged = pd.merge(df_ventas_historic_cluster_final_merged, _df_ventas_historic_cluster_with_sq, on='cod_tienda')
df_ventas_historic_cluster_final_merged

,cod_tienda,promocion_cabecera,promocion_descuento,promocion_expositor,promocion_extra_cantidad,promocion_folleto,promocion_isla,promocion_multicompra,promocion_regalo,factor_extrapolacion,cod_canal,national_holidays_2021,regional_holidays_2021,local_holidays_2021,national_holidays_2022,regional_holidays_2022,local_holidays_2022,national_holidays_2023,regional_holidays_2023,local_holidays_2023,SEGMENT_ARTISANS_x,SEGMENT_BLACK_x,SEGMENT_CELIACS_x,SEGMENT_EXTRA_x,SEGMENT_FLAVOURED_x,SEGMENT_IMPORT PREMIUM_x,SEGMENT_IMPORT SPECIAL_x,SEGMENT_NATIONAL_x,SEGMENT_WITHOUT ALCOHOL_x,SEGMENT_ZERO ALCOHOL_x,...,MAN 2 - BRAND 3,MAN 2 - BRAND 4,MAN 2 - BRAND 5,MAN 2 - OTHER BRANDS,MAN 3 - BRAND 1,MAN 3 - BRAND 2,MAN 3 - BRAND 3,MAN 3 - BRAND 4,MAN 3 - BRAND 5,MAN 3 - OTHER BRANDS,MAN 4 - BRAND 1,MAN 5 - BRAND 1,MAN 5 - BRAND 2,MAN 5 - OTHER BRANDS,MAN 6 - BRAND 1,OTHER MAN - OTHER BRANDS,SEGMENT_ARTISANS_y,SEGMENT_BLACK_y,SEGMENT_CELIACS_y,SEGMENT_EXTRA_y,SEGMENT_FLAVOURED_y,SEGMENT_IMPORT PREMIUM_y,SEGMENT_IMPORT SPECIAL_y,SEGMENT_NATIONAL_y,SEGMENT_WITHOUT ALCOHOL_y,SEGMENT_ZERO ALCOHOL_y,sales_surface_sqmeters,ventas_valor_per_store,Channel,revenue_per_sqmeter
0,1,1,1,1,1,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,...,34296.4296,381.2744,2789.4838,78053.2296,9.120631e+04,NaN,7.970274e+04,1.489422e+05,NaN,14577.3146,NaN,NaN,1213.6000,NaN,7.044279e+05,12227.5290,NaN,1414.6977,421.6480,7.561654e+04,27506.3214,5.310461e+05,3305.1734,6.300099e+05,71392.9384,125118.5008,500,24415.94,Supermarkets,48.831880
1,2,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,8.159604e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.159604e+02,NaN,NaN,NaN,NaN,NaN,NaN,500,8.52,Supermarkets,0.017040
2,3,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,896062.5975,38137.5001,79546.7018,364070.1169,6.454752e+05,NaN,3.162244e+06,1.942609e+06,NaN,102719.8487,1.586968e+06,82.2500,90054.6162,NaN,1.083251e+05,852118.1660,181071.9192,126202.5774,62344.3673,2.056731e+06,678890.3149,1.315005e+06,391735.2451,7.045113e+06,369141.4429,479443.2445,1200,174172.84,Supermarkets,145.144033
3,4,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,141834.0120,11071.3784,15241.1026,41484.0975,5.562945e+04,832.7146,1.199536e+05,2.091851e+05,NaN,28714.5962,1.638660e+05,695.7223,12907.7715,51.7644,5.221909e+04,287779.6831,22195.1685,48411.6924,4216.3742,2.091155e+05,77684.2095,2.629581e+05,150471.2403,5.378297e+05,40797.3675,57624.5116,2200,629539.99,Supermarkets,286.154541
4,5,0,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,...,94031.3046,NaN,41703.5397,64664.9017,1.145855e+05,NaN,1.315183e+05,5.700949e+05,NaN,NaN,7.320569e+04,NaN,2804.5336,NaN,1.318352e+06,16696.9154,NaN,NaN,2804.5336,1.216046e+05,79013.7463,6.986261e+05,41703.5397,1.931208e+06,143889.3444,112627.8304,900,151578.07,Supermarkets,168.420078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,553,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,765827.1698,15288.9601,77586.9689,238446.1472,1.234292e+06,1459.1147,3.424622e+06,1.043402e+06,NaN,299899.9401,4.280039e+05,82.4850,78140.7906,2751.5047,4.019239e+06,333681.4539,34137.5301,64338.3038,44759.6194,7.102494e+05,506361.3887,4.687378e+06,275582.2222,8.084835e+06,554765.9192,579462.9100,10300,3370265.24,Hipermarkets,327.210217
539,554,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,...,628907.9155,2764.2759,46065.5712,183230.7452,8.065940e+05,NaN,9.688083e+05,4.404443e+05,NaN,110550.7681,3.747018e+05,NaN,NaN,NaN,2.226328e+06,96917.2022,NaN,10115.7618,25409.1593,3.395601e+05,143621.9785,2.820922e+06,95464.0766,4.283443e+06,475089.6387,344571.2952,900,378730.10,Supermarkets,420.811222
540,555,0,1,1,0,1,1,0,1,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,...,120953.9098,13399.158

In [67]:
df_ventas_historic_cluster_final_merged = pd.merge(df_ventas_historic_cluster_final_merged, provinces, on='cod_tienda')

In [68]:
df_ventas_historic_cluster_final_merged

,cod_tienda,promocion_cabecera,promocion_descuento,promocion_expositor,promocion_extra_cantidad,promocion_folleto,promocion_isla,promocion_multicompra,promocion_regalo,factor_extrapolacion,cod_canal,national_holidays_2021,regional_holidays_2021,local_holidays_2021,national_holidays_2022,regional_holidays_2022,local_holidays_2022,national_holidays_2023,regional_holidays_2023,local_holidays_2023,SEGMENT_ARTISANS_x,SEGMENT_BLACK_x,SEGMENT_CELIACS_x,SEGMENT_EXTRA_x,SEGMENT_FLAVOURED_x,SEGMENT_IMPORT PREMIUM_x,SEGMENT_IMPORT SPECIAL_x,SEGMENT_NATIONAL_x,SEGMENT_WITHOUT ALCOHOL_x,SEGMENT_ZERO ALCOHOL_x,...,MAN 2 - BRAND 4,MAN 2 - BRAND 5,MAN 2 - OTHER BRANDS,MAN 3 - BRAND 1,MAN 3 - BRAND 2,MAN 3 - BRAND 3,MAN 3 - BRAND 4,MAN 3 - BRAND 5,MAN 3 - OTHER BRANDS,MAN 4 - BRAND 1,MAN 5 - BRAND 1,MAN 5 - BRAND 2,MAN 5 - OTHER BRANDS,MAN 6 - BRAND 1,OTHER MAN - OTHER BRANDS,SEGMENT_ARTISANS_y,SEGMENT_BLACK_y,SEGMENT_CELIACS_y,SEGMENT_EXTRA_y,SEGMENT_FLAVOURED_y,SEGMENT_IMPORT PREMIUM_y,SEGMENT_IMPORT SPECIAL_y,SEGMENT_NATIONAL_y,SEGMENT_WITHOUT ALCOHOL_y,SEGMENT_ZERO ALCOHOL_y,sales_surface_sqmeters,ventas_valor_per_store,Channel,revenue_per_sqmeter,Provincia
0,1,1,1,1,1,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,...,381.2744,2789.4838,78053.2296,9.120631e+04,NaN,7.970274e+04,1.489422e+05,NaN,14577.3146,NaN,NaN,1213.6000,NaN,7.044279e+05,12227.5290,NaN,1414.6977,421.6480,7.561654e+04,27506.3214,5.310461e+05,3305.1734,6.300099e+05,71392.9384,125118.5008,500,24415.94,Supermarkets,48.831880,Almeria
1,2,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,NaN,NaN,NaN,8.159604e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.159604e+02,NaN,NaN,NaN,NaN,NaN,NaN,500,8.52,Supermarkets,0.017040,Area Metropolitana de Madrid
2,3,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,38137.5001,79546.7018,364070.1169,6.454752e+05,NaN,3.162244e+06,1.942609e+06,NaN,102719.8487,1.586968e+06,82.2500,90054.6162,NaN,1.083251e+05,852118.1660,181071.9192,126202.5774,62344.3673,2.056731e+06,678890.3149,1.315005e+06,391735.2451,7.045113e+06,369141.4429,479443.2445,1200,174172.84,Supermarkets,145.144033,Cantabria
3,4,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,11071.3784,15241.1026,41484.0975,5.562945e+04,832.7146,1.199536e+05,2.091851e+05,NaN,28714.5962,1.638660e+05,695.7223,12907.7715,51.7644,5.221909e+04,287779.6831,22195.1685,48411.6924,4216.3742,2.091155e+05,77684.2095,2.629581e+05,150471.2403,5.378297e+05,40797.3675,57624.5116,2200,629539.99,Supermarkets,286.154541,Vizcaya
4,5,0,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,...,NaN,41703.5397,64664.9017,1.145855e+05,NaN,1.315183e+05,5.700949e+05,NaN,NaN,7.320569e+04,NaN,2804.5336,NaN,1.318352e+06,16696.9154,NaN,NaN,2804.5336,1.216046e+05,79013.7463,6.986261e+05,41703.5397,1.931208e+06,143889.3444,112627.8304,900,151578.07,Supermarkets,168.420078,Castellon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,553,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,15288.9601,77586.9689,238446.1472,1.234292e+06,1459.1147,3.424622e+06,1.043402e+06,NaN,299899.9401,4.280039e+05,82.4850,78140.7906,2751.5047,4.019239e+06,333681.4539,34137.5301,64338.3038,44759.6194,7.102494e+05,506361.3887,4.687378e+06,275582.2222,8.084835e+06,554765.9192,579462.9100,10300,3370265.24,Hipermarkets,327.210217,Albacete
539,554,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,...,2764.2759,46065.5712,183230.7452,8.065940e+05,NaN,9.688083e+05,4.404443e+05,NaN,110550.7681,3.747018e+05,NaN,NaN,NaN,2.226328e+06,96917.2022,NaN,10115.7618,25409.1593,3.395601e+05,143621.9785,2.820922e+06,95464.0766,4.283443e+06,475089.6387,344571.2952,900,378730.10,Supermarkets,420.811222,Alicante
540,555,0,1,1,0,1,1,0,1,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,...,13399.1583,18796.7

In [69]:

df_ventas_historic_cluster_final_merged = pd.merge(df_ventas_historic_cluster_final_merged, macrodata_cleaned, on='cod_tienda',how='left')

In [70]:
df_ventas_historic_cluster_final_merged

,cod_tienda,promocion_cabecera,promocion_descuento,promocion_expositor,promocion_extra_cantidad,promocion_folleto,promocion_isla,promocion_multicompra,promocion_regalo,factor_extrapolacion,cod_canal,national_holidays_2021,regional_holidays_2021,local_holidays_2021,national_holidays_2022,regional_holidays_2022,local_holidays_2022,national_holidays_2023,regional_holidays_2023,local_holidays_2023,SEGMENT_ARTISANS_x,SEGMENT_BLACK_x,SEGMENT_CELIACS_x,SEGMENT_EXTRA_x,SEGMENT_FLAVOURED_x,SEGMENT_IMPORT PREMIUM_x,SEGMENT_IMPORT SPECIAL_x,SEGMENT_NATIONAL_x,SEGMENT_WITHOUT ALCOHOL_x,SEGMENT_ZERO ALCOHOL_x,...,MAN 3 - BRAND 4,MAN 3 - BRAND 5,MAN 3 - OTHER BRANDS,MAN 4 - BRAND 1,MAN 5 - BRAND 1,MAN 5 - BRAND 2,MAN 5 - OTHER BRANDS,MAN 6 - BRAND 1,OTHER MAN - OTHER BRANDS,SEGMENT_ARTISANS_y,SEGMENT_BLACK_y,SEGMENT_CELIACS_y,SEGMENT_EXTRA_y,SEGMENT_FLAVOURED_y,SEGMENT_IMPORT PREMIUM_y,SEGMENT_IMPORT SPECIAL_y,SEGMENT_NATIONAL_y,SEGMENT_WITHOUT ALCOHOL_y,SEGMENT_ZERO ALCOHOL_y,sales_surface_sqmeters,ventas_valor_per_store,Channel,revenue_per_sqmeter,Provincia_x,Sectors,cod_provincia,Provincia_y,Net income per person,year,Income
0,1,1,1,1,1,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,...,1.489422e+05,NaN,14577.3146,NaN,NaN,1213.6000,NaN,7.044279e+05,12227.5290,NaN,1414.6977,421.6480,7.561654e+04,27506.3214,5.310461e+05,3305.1734,6.300099e+05,71392.9384,125118.5008,500,24415.94,Supermarkets,48.831880,Almeria,Primary Sector,4,Almeria,Renta neta media por persona,2020,9709
1,2,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.159604e+02,NaN,NaN,NaN,NaN,NaN,NaN,500,8.52,Supermarkets,0.017040,Area Metropolitana de Madrid,Metropolis,95,Madrid,Renta neta media por persona,2020,15579
2,3,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1.942609e+06,NaN,102719.8487,1.586968e+06,82.2500,90054.6162,NaN,1.083251e+05,852118.1660,181071.9192,126202.5774,62344.3673,2.056731e+06,678890.3149,1.315005e+06,391735.2451,7.045113e+06,369141.4429,479443.2445,1200,174172.84,Supermarkets,145.144033,Cantabria,Industrial,39,Cantabria,Renta neta media por persona,2020,12865
3,4,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,2.091851e+05,NaN,28714.5962,1.638660e+05,695.7223,12907.7715,51.7644,5.221909e+04,287779.6831,22195.1685,48411.6924,4216.3742,2.091155e+05,77684.2095,2.629581e+05,150471.2403,5.378297e+05,40797.3675,57624.5116,2200,629539.99,Supermarkets,286.154541,Vizcaya,Industrial,48,Vizcaya,Renta neta media por persona,2020,15555
4,5,0,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,...,5.700949e+05,NaN,NaN,7.320569e+04,NaN,2804.5336,NaN,1.318352e+06,16696.9154,NaN,NaN,2804.5336,1.216046e+05,79013.7463,6.986261e+05,41703.5397,1.931208e+06,143889.3444,112627.8304,900,151578.07,Supermarkets,168.420078,Castellon,Primary Sector,12,Castellon,Renta neta media por persona,2020,12079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,553,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1.043402e+06,NaN,299899.9401,4.280039e+05,82.4850,78140.7906,2751.5047,4.019239e+06,333681.4539,34137.5301,64338.3038,44759.6194,7.102494e+05,506361.3887,4.687378e+06,275582.2222,8.084835e+06,554765.9192,579462.9100,10300,3370265.24,Hipermarkets,327.210217,Albacete,Industrial,2,Albacete,Renta neta media por persona,2020,11104
539,554,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,...,4.404443e+05,NaN,110550.7681,3.747018e+05,NaN,NaN,NaN,2.226328e+06,96917.2022,NaN,10115.7618,25409.1593,3.395601e+05,143621.9785,2.820922e+06,95464.0766,4.283443e+06,475089.6387,344571.2952,900,378730.10,Supermarkets,420.811222,Alicante,touristic,3,Alicante,Renta neta media por persona,2020,14962
540,555,0,1,1,0,1,1,0,1,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,...,2.658553e+05,NaN,3815

In [71]:
df_ventas_historic_cluster_final_merged=df_ventas_historic_cluster_final_merged.fillna(0)

In [72]:
df_ventas_historic_cluster_final_merged

,cod_tienda,promocion_cabecera,promocion_descuento,promocion_expositor,promocion_extra_cantidad,promocion_folleto,promocion_isla,promocion_multicompra,promocion_regalo,factor_extrapolacion,cod_canal,national_holidays_2021,regional_holidays_2021,local_holidays_2021,national_holidays_2022,regional_holidays_2022,local_holidays_2022,national_holidays_2023,regional_holidays_2023,local_holidays_2023,SEGMENT_ARTISANS_x,SEGMENT_BLACK_x,SEGMENT_CELIACS_x,SEGMENT_EXTRA_x,SEGMENT_FLAVOURED_x,SEGMENT_IMPORT PREMIUM_x,SEGMENT_IMPORT SPECIAL_x,SEGMENT_NATIONAL_x,SEGMENT_WITHOUT ALCOHOL_x,SEGMENT_ZERO ALCOHOL_x,...,MAN 3 - BRAND 4,MAN 3 - BRAND 5,MAN 3 - OTHER BRANDS,MAN 4 - BRAND 1,MAN 5 - BRAND 1,MAN 5 - BRAND 2,MAN 5 - OTHER BRANDS,MAN 6 - BRAND 1,OTHER MAN - OTHER BRANDS,SEGMENT_ARTISANS_y,SEGMENT_BLACK_y,SEGMENT_CELIACS_y,SEGMENT_EXTRA_y,SEGMENT_FLAVOURED_y,SEGMENT_IMPORT PREMIUM_y,SEGMENT_IMPORT SPECIAL_y,SEGMENT_NATIONAL_y,SEGMENT_WITHOUT ALCOHOL_y,SEGMENT_ZERO ALCOHOL_y,sales_surface_sqmeters,ventas_valor_per_store,Channel,revenue_per_sqmeter,Provincia_x,Sectors,cod_provincia,Provincia_y,Net income per person,year,Income
0,1,1,1,1,1,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,...,1.489422e+05,0.0,14577.3146,0.000000e+00,0.0000,1213.6000,0.0000,7.044279e+05,12227.5290,0.0000,1414.6977,421.6480,7.561654e+04,27506.3214,5.310461e+05,3305.1734,6.300099e+05,71392.9384,125118.5008,500,24415.94,Supermarkets,48.831880,Almeria,Primary Sector,4,Almeria,Renta neta media por persona,2020,9709
1,2,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0.000000e+00,0.0,0.0000,0.000000e+00,0.0000,0.0000,0.0000,0.000000e+00,0.0000,0.0000,0.0000,0.0000,8.159604e+02,0.0000,0.000000e+00,0.0000,0.000000e+00,0.0000,0.0000,500,8.52,Supermarkets,0.017040,Area Metropolitana de Madrid,Metropolis,95,Madrid,Renta neta media por persona,2020,15579
2,3,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1.942609e+06,0.0,102719.8487,1.586968e+06,82.2500,90054.6162,0.0000,1.083251e+05,852118.1660,181071.9192,126202.5774,62344.3673,2.056731e+06,678890.3149,1.315005e+06,391735.2451,7.045113e+06,369141.4429,479443.2445,1200,174172.84,Supermarkets,145.144033,Cantabria,Industrial,39,Cantabria,Renta neta media por persona,2020,12865
3,4,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,2.091851e+05,0.0,28714.5962,1.638660e+05,695.7223,12907.7715,51.7644,5.221909e+04,287779.6831,22195.1685,48411.6924,4216.3742,2.091155e+05,77684.2095,2.629581e+05,150471.2403,5.378297e+05,40797.3675,57624.5116,2200,629539.99,Supermarkets,286.154541,Vizcaya,Industrial,48,Vizcaya,Renta neta media por persona,2020,15555
4,5,0,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,...,5.700949e+05,0.0,0.0000,7.320569e+04,0.0000,2804.5336,0.0000,1.318352e+06,16696.9154,0.0000,0.0000,2804.5336,1.216046e+05,79013.7463,6.986261e+05,41703.5397,1.931208e+06,143889.3444,112627.8304,900,151578.07,Supermarkets,168.420078,Castellon,Primary Sector,12,Castellon,Renta neta media por persona,2020,12079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,553,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1.043402e+06,0.0,299899.9401,4.280039e+05,82.4850,78140.7906,2751.5047,4.019239e+06,333681.4539,34137.5301,64338.3038,44759.6194,7.102494e+05,506361.3887,4.687378e+06,275582.2222,8.084835e+06,554765.9192,579462.9100,10300,3370265.24,Hipermarkets,327.210217,Albacete,Industrial,2,Albacete,Renta neta media por persona,2020,11104
539,554,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,...,4.404443e+05,0.0,110550.7681,3.747018e+05,0.0000,0.0000,0.0000,2.226328e+06,96917.2022,0.0000,10115.7618,25409.1593,3.395601e+05,143621.9785,2.820922e+06,95464.0766,4.283443e+06,475089.6387,344571.2952,900,378730.10,Supermarkets,420.811222,Alicante,touristic,3,Alicante,Re

In [ ]:
#df_ventas_historic_cluster_final_merged.to_csv('Datasets/_df_ventas_historic_cluster_final_merged.csv')